In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from curve import Curve, curvePlot
import quadrature4
import quadrature_minimal
from copy import deepcopy
import json
DTYPE="float128"

In [ ]:
points = []
resolution = 5
for i in range(resolution):
    theta = 2 * np.pi / resolution * i
    points.append(np.array([np.cos(theta), np.sin(theta), 0], dtype=DTYPE))
easy_square = Curve(points)
curvePlot(easy_square, False)


In [ ]:
# Gradient Flow
deltaT=0.01
Lambda=0.01
M=30000
qPlot=False
qGraphic=True
PLOTFREQ = 100
curve = easy_square
xpoints = []
ypoints = []
zpoints = []
for t in range(M):
    #print(f"{t} / {M} ({t/M*100:.2f}%)")
    curve = curve - (quadrature_minimal.dEnergy(curve, 3, 6) + curve * Lambda) * deltaT
    if (t % PLOTFREQ == 0):
        if qGraphic:
            curvePlot(curve, False, size=(-10,10), title=f"{t} / {M} ({t/M*100:.2f}%)")
        if qPlot:
            xpointAtTime = []
            ypointAtTime = []
            zpointAtTime = []
            for i in range(curve.J):
                xpointAtTime.append(curve[i][0])
                ypointAtTime.append(curve[i][1])
                zpointAtTime.append(curve[i][2])
            xpoints.append(xpointAtTime)
            ypoints.append(ypointAtTime)
            zpoints.append(zpointAtTime)

if qPlot:
    with open("x.json","w") as file:
        json.dump(xpoints, file)
    with open("y.json","w") as file:
        json.dump(ypoints, file)
    with open("z.json","w") as file:
        json.dump(zpoints, file)

In [ ]:
quadrature4.derivative_index(1,5)

In [ ]:
#easy_square = Curve([
#    np.array([4.16, -2.86, 0], dtype="float64"),
#    np.array([5.74, 2.0, 0], dtype="float64"),
#    np.array([-3.34, 0.88, 0], dtype="float64"),
#    np.array([-3, -2, 0], dtype="float64")
#    ]
#)
#curvePlot(easy_square)

In [ ]:
# Lab
easy_square_plus = deepcopy(easy_square)
easy_square_minus = deepcopy(easy_square)
perturbation=0.00001
ALPHA=2
BETA=4
derivative_function=quadrature4.ijk
p = 3
q = 0
r = 1
k = 1
easy_square_plus[k] += np.array([perturbation, 0, 0])
easy_square_minus[k] += np.array([-perturbation, 0, 0])
#easy_square_plus[k] = np.array([0, perturbation, 0])
#easy_square_minus[k] = np.array([0, -perturbation, 0])
xi, eta, dxi, deta = derivative_function(easy_square, p, q, r, ALPHA, BETA)
#(dxi, deta)


In [ ]:
# Derivative of summand
xiEdgeLen = np.linalg.norm(easy_square[p+1] - easy_square[p])
xjEdgeLen = np.linalg.norm(easy_square[q+1] - easy_square[q])
dsummand = quadrature4.dkij(easy_square, p, q, k, ALPHA, BETA) * xiEdgeLen * xjEdgeLen
dsummand += quadrature4.kij(easy_square, p, q, ALPHA, BETA) * quadrature4.dProductOfLengths(easy_square, p, q, k)

approxDSummand = (quadrature4.kij(easy_square_plus, p, q, ALPHA, BETA) * np.linalg.norm(easy_square_plus[p+1] - easy_square_plus[p]) * np.linalg.norm(easy_square_plus[q+1] - easy_square_plus[q])
                   - quadrature4.kij(easy_square_minus, p, q, ALPHA, BETA) * np.linalg.norm(easy_square_minus[p+1] - easy_square_minus[p]) * np.linalg.norm(easy_square_minus[q+1] - easy_square_minus[q])
                   ) / (2*perturbation)

(dsummand, approxDSummand)


In [ ]:
# derivative kij
(
    (quadrature4.kij(easy_square_plus, p, q, ALPHA, BETA) - quadrature4.kij(easy_square_minus, p, q, ALPHA, BETA)) / (2*perturbation), 
quadrature4.dkij(easy_square, p, q, k, alpha=ALPHA, beta=BETA)
)

In [ ]:
# Comparison of kalphabeta values
xi, eta, dxi, deta = derivative_function(easy_square, p, q, r, ALPHA, BETA)
referenceValue = quadrature4.kernelalphabeta(easy_square[p], easy_square[q], (easy_square[r] - easy_square[r+1]) / (np.linalg.norm(easy_square[r] - easy_square[r+1])), alpha=ALPHA, beta=BETA)
newValue = xi**(ALPHA/2) / eta
(referenceValue, newValue)

In [ ]:
xip, etap, dxip, detap = derivative_function(easy_square_plus, p, q, r, ALPHA, BETA)
xin, etan, dxin, detan = derivative_function(easy_square_minus, p, q, r, ALPHA, BETA)
((xip - xin) / (2*perturbation), (etap - etan) / (2*perturbation))

In [ ]:
quadrature4.dEnergy(easy_square, ALPHA, BETA).list_of_points

In [ ]:
quadrature_minimal.dEnergy(easy_square, ALPHA, BETA).list_of_points

In [ ]:
for i in range(len(easy_square)):
    print(quadrature4.curveDifferential(easy_square, i, np.array([perturbation, 0, 0]), quadrature4.energy) / perturbation)